In [10]:
import numpy as np

In [11]:
# Example matrix
matrix = np.array([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9]])

# Print upper triangular matrix using list comprehension
upper_triangular = [[matrix[i, j] for j in range(i, matrix.shape[1])] for i in range(matrix.shape[0])]
for row in upper_triangular:
    print(row)


[1, 2, 3]
[5, 6]
[9]


In [12]:
# Sample matrix
matrix = np.array([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9]])

# Print upper triangular matrix with values in the main diagonal multiplied by 0.5
upper_triangular = [[matrix[i, j] * 0.5 if i == j else matrix[i, j] for j in range(i, matrix.shape[1])] for i in range(matrix.shape[0])]

for row in upper_triangular:
    print(row)


[0.5, 2, 3]
[2.5, 6]
[4.5]


In [13]:
# Covariance matrix
covariance_matrix = np.array([[0.1, 0.02, 0.05, 0.03],
                               [0.02, 0.1, 0.03, 0.04],
                               [0.05, 0.03, 0.15, 0.07],
                               [0.03, 0.04, 0.07, 0.2]])  

In [14]:
# Extract upper triangular elements of covariance matrix
upper_triangular_covariance = np.triu(covariance_matrix)

upper_triangular_covariance

array([[0.1 , 0.02, 0.05, 0.03],
       [0.  , 0.1 , 0.03, 0.04],
       [0.  , 0.  , 0.15, 0.07],
       [0.  , 0.  , 0.  , 0.2 ]])

#### Entire Cov. Matrix

In [22]:
%%time
from docplex.mp.model import Model
import numpy as np

# Define the data - Expected returns of assets
returns = np.array([0.1, 0.15, 0.08, 0.12])  

# Number of assets
num_assets = len(returns)

# Covariance matrix
covariance_matrix = np.array([[0.1, 0.02, 0.05, 0.03],
                              [0.02, 0.1, 0.03, 0.04],
                              [0.05, 0.03, 0.15, 0.07],
                              [0.03, 0.04, 0.07, 0.2]])  # Covariance matrix

# Create a model
model = Model(name='Markowitz_Portfolio')

# Define variables
x = {i: model.continuous_var(name='x{}'.format(i)) for i in range(num_assets)}

# Define objective function (minimize portfolio risk)
objective = model.sum(covariance_matrix[i, j] * x[i] * x[j] for i in range(covariance_matrix.shape[0]) for j in range(covariance_matrix.shape[1]))

model.minimize(objective)

# Define constraints
model.add_constraint(model.sum(x[i] for i in range(num_assets)) == 1)  # Total investment equals 100%
model.add_constraints(x[i] >= 0 for i in range(num_assets))  # No short selling

# Solve the model
solution = model.solve()

# Display results
if solution:
    optimal_portfolio = {i: solution.get_value(x[i]) for i in range(num_assets)}
    print(f'Model Objective value   :{model.objective_value}')
    print("Optimal Portfolio Weights:")
    for i in range(num_assets):
        print("Asset {}: {:.2f}%".format(i, optimal_portfolio[i] * 100))
    print()
else:
    print("No solution found.")


Model Objective value   :0.05527256478999118
Optimal Portfolio Weights:
Asset 0: 38.61%
Asset 1: 40.21%
Asset 2: 11.35%
Asset 3: 9.83%

CPU times: total: 0 ns
Wall time: 12 ms


#### Upper Triangular Cov. Matrix

In [23]:
%%time
from docplex.mp.model import Model
import numpy as np

# Number of assets
num_assets = len(returns)

# Define the data - Expected returns of assets
returns = np.array([0.1, 0.15, 0.08, 0.12])  

# Covariance matrix
covariance_matrix = np.array([[0.1, 0.02, 0.05, 0.03],
                              [0.02, 0.1, 0.03, 0.04],
                              [0.05, 0.03, 0.15, 0.07],
                              [0.03, 0.04, 0.07, 0.2]])  # Covariance matrix

num_assets = len(returns)

# Create a model
model = Model(name='Markowitz_Portfolio')

# Define variables
x = {i: model.continuous_var(name='x{}'.format(i)) for i in range(num_assets)}

# Define objective function (minimize portfolio risk)
objective = model.sum(0.5 * covariance_matrix[i, j] * x[i] * x[j] if i==j else covariance_matrix[i, j] * x[i] * x[j] for i in range(covariance_matrix.shape[0]) for j in range(i, covariance_matrix.shape[1]))

model.minimize(objective)

# Define constraints
model.add_constraint(model.sum(x[i] for i in range(num_assets)) == 1)  # Total investment equals 100%
model.add_constraints(x[i] >= 0 for i in range(num_assets))  # No short selling

# Solve the model
solution = model.solve()

# Display results
if solution:
    optimal_portfolio = {i: solution.get_value(x[i]) for i in range(num_assets)}
    print(f'Model Objective value   :{model.objective_value}')
    print("Optimal Portfolio Weights:")
    for i in range(num_assets):
        print("Asset {}: {:.2f}%".format(i, optimal_portfolio[i] * 100))
    print()
else:
    print("No solution found.")


Model Objective value   :0.027636282394995642
Optimal Portfolio Weights:
Asset 0: 38.61%
Asset 1: 40.21%
Asset 2: 11.35%
Asset 3: 9.83%

CPU times: total: 15.6 ms
Wall time: 13 ms
